In [91]:
%matplotlib inline

In [18]:
from __future__ import division
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from math import sqrt
from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
import matplotlib.pyplot as plt

LEARNING_RATE = .1
PCA_COMPONENTS = 80
EPOCHS = 15
# color = true keeps color channels seperate
people = fetch_lfw_people(color=True, resize=0.25, slice_=(slice(61,189), slice(61,189)))

X = people.images
y = np.asarray(people.target, dtype=np.int32)

labels = people.target_names
n_classes = labels.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

In [20]:
print labels[0]

AJ Cook


We define the model as such 

In [21]:
def model(features, labels, mode):
    """Model function for CNN."""
  
    # Input Layer
    input_layer = tf.convert_to_tensor(features)

    # Convolutional Layers 1 + 2
    # Computes 32 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 32, 32, 3]
    # Output Tensor Shape: [batch_size, 32, 32, 32]
    # NOTE: tf.layers.conv2d input MUST be floating point for some reason???
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    conv2 = tf.layers.conv2d(
      inputs= conv1,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 32, 32, 32]
    # Output Tensor Shape: [batch_size, 16, 16, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # add dropout layer to introduce randomness / reduce possibility of overfitting
    dropout1 = tf.layers.dropout(
    inputs=pool1, rate=0.25, training=mode == learn.ModeKeys.TRAIN)

    # Convolutional Layers 3 + 4
    # Computes 64 features using a 3x3 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 16, 16, 32]
    # Output Tensor Shape: [batch_size, 16, 16, 64]
    conv3 = tf.layers.conv2d(
      inputs= dropout1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

    conv4 = tf.layers.conv2d(
      inputs= conv3,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 16, 16, 64]
    # Output Tensor Shape: [batch_size, 8, 8, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)

    # add dropout layer to introduce randomness / reduce possibility of overfitting
    dropout2 = tf.layers.dropout(
      inputs=pool2, rate=0.25, training=mode == learn.ModeKeys.TRAIN)

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 16, 16, 64]
    # Output Tensor Shape: [batch_size, 4096]
    pool3_flat = tf.reshape(dropout2, [-1, 8 * 8 * 64])

    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 4096]
    # Output Tensor Shape: [batch_size, 1024]
    dense1 = tf.layers.dense(inputs=pool3_flat, units=1024, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout3 = tf.layers.dropout(
      inputs=dense1, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

    # add a 2nd dense layer with 512 neurons
    dense2 = tf.layers.dense(inputs= dropout3, units=512, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout4 = tf.layers.dropout(
      inputs=dense2, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 512]
    # Output Tensor Shape: [batch_size, n_classes]
    logits = tf.layers.dense(inputs=dropout4, units=n_classes)

    loss = None
    train_op = None

    # Calculate Loss (for both TRAIN and EVAL modes)
    # !!! Make sure depth variable is set properly relative to data !!!!
    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=n_classes)
        loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    # Use Adagrad optimizer + initial learning rate of 0.001  
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=0.0001, 
            optimizer="Adam")

    # Generate Predictions
    predictions = {
      "classes": tf.argmax(
          input=logits, axis=1),
      "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor")
    }

    # Return a ModelFnOps object
    return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [22]:
lfw_classifier = learn.Estimator(
    model_fn=model,
    model_dir='./cnn/lfw-data-1'
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13762e450>, '_model_dir': './cnn/lfw-data-1', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': '', '_master': ''}


In [23]:
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

In [24]:
# Train the model
with tf.device('/cpu:0'):
    lfw_classifier.fit(
          x=X_train,
          y=y_train,
          batch_size=64,
          steps=5100,
          monitors=[logging_hook])

# Configure the accuracy metric for evaluation
metrics = {
  "accuracy":
      # learn.metric_spec.MetricSpec(
      learn.MetricSpec(
          metric_fn=tf.metrics.accuracy, prediction_key="classes"),
}

# remove training data from memory
#  del train_data
#  del train_labels
del X
del y

# eval half of the training data
small_train =  np.array_split(X_train, 2)[1]
small_labs =  np.array_split(y_train, 2)[1]
eval_results = lfw_classifier.evaluate(
  x= small_train, y=small_labs, metrics=metrics)
print(eval_results)

####################################
# Evaluate all test data and print results
eval_results = lfw_classifier.evaluate(
x=X_test, y=y_test, metrics=metrics)
print(eval_results)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./cnn/lfw-data-1/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
INFO:tensorflow:loss = 54.2008, step = 1
INFO:tensorflow:probabilities = [[ 0.00015538  0.0003284   0.00011346 ...,  0.00006442  0.00011011
   0.00004811]
 [ 0.00032377  0.0001434   0.00008736 ...,  0.00011483  0.00097685
   0.00019197]
 [ 0.00014359  0.00025863  0.00009482 ...,  0.00009677  0.00019441
   0.00006377]
 ..., 
 [ 0.00013094  0.00035259  0.00015871 ...,  0.00011818  0.0002111
   0.00018698]
 [ 0.00013951  0.00033697  0.00034985 ...,  0.00004522  0.00016213
   0.00019986]
 [ 0.00027111  0.00009147  0.00006636 ...,  0.00008429  0.00019631
   0.00015647]] (44.493 sec)
INFO:tensorflow:global_step/sec: 1.14514
INFO:tensorflow:probabi

KeyboardInterrupt: 

Beginning epoch 0
